## Импорт необходимых библиотек

In [1]:
import cv2
import numpy as np
import os
import logging
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [ ]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

## Функции предобработки изображений

In [2]:
def cluster_density(cluster_mask):
    logger.debug("Calculating cluster density.")
    contours, _ = cv2.findContours(cluster_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(contour) for contour in contours]
    total_area = sum(areas)
    logger.debug(f"Total area calculated: {total_area}")
    return total_area

In [3]:
def combine_vertical_contours(contours):
    logger.debug("Combining vertical contours.")
    combined_contours = []
    skip = set()

    for i, cnt1 in enumerate(contours):
        if i in skip:
            continue

        x1, y1, w1, h1 = cv2.boundingRect(cnt1)
        combined_rect = [x1, y1, x1 + w1, y1 + h1]  # [left, top, right, bottom]

        for j, cnt2 in enumerate(contours):
            if i == j or j in skip:
                continue

            x2, y2, w2, h2 = cv2.boundingRect(cnt2)
            if (x1 <= x2 <= x1 + w1) or (x2 <= x1 <= x2 + w2):  # Check vertical overlap
                combined_rect[0] = min(combined_rect[0], x2)
                combined_rect[1] = min(combined_rect[1], y2)
                combined_rect[2] = max(combined_rect[2], x2 + w2)
                combined_rect[3] = max(combined_rect[3], y2 + h2)
                skip.add(j)

        combined_contours.append(combined_rect)
        logger.debug(f"Combined rect: {combined_rect}")

    return combined_contours

In [4]:
def preprocess_image(path):
    logger.info(f"Preprocessing image: {path}")
    image = cv2.imread(path)
    if image is None:
        logger.error(f"Failed to read image from path: {path}")
        return []

    logger.debug("Applying sharpening filter.")
    sharp_filter = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    image = cv2.filter2D(image, -1, kernel=sharp_filter)

    logger.debug("Reshaping and converting image to float32.")
    pixels = image.reshape(-1, 3)
    pixels = np.float32(pixels)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    k = 5
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    centers = np.uint8(centers)
    segmented_image = centers[labels.flatten()]
    segmented_image = segmented_image.reshape(image.shape)

    logger.debug("Calculating densities for each cluster.")
    densities = []
    for i in range(k):
        cluster_mask = (labels == i).reshape(image.shape[:2]).astype(np.uint8) * 255
        density = cluster_density(cluster_mask)
        densities.append(density)
    selected_cluster = np.argmin(densities)
    logger.debug(f"Selected cluster: {selected_cluster}")

    mask = (labels == selected_cluster).reshape(image.shape[:2])
    masked_image = cv2.bitwise_and(image, image, mask=mask.astype(np.uint8) * 255)

    logger.debug("Creating binary image.")
    binary_image = cv2.threshold(cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    binary_image = cv2.bitwise_not(binary_image)
    binary_image = cv2.erode(binary_image, np.ones((2, 2), np.uint8))

    logger.debug("Finding contours.")
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])
    combined_contours = combine_vertical_contours(contours)

    captcha_h = max(map(lambda c: c[3] - c[1], combined_contours))
    symbols = []

    logger.debug("Processing each contour to extract symbols.")
    for x1, y1, x2, y2 in combined_contours:
        symbol = binary_image[y1:y2, x1:x2]
        symbol = cv2.dilate(symbol, np.ones((2, 2), np.uint8))
        h = int(30 / captcha_h * (y2 - y1))
        w = int(h * ((x2 - x1) / (y2 - y1)))
        w = min(37, max(1, w))  # Ensure width is at least 1
        h = max(1, h)  # Ensure height is at least 1

        logger.debug(f"Resizing symbol: target size ({w}, {h})")
        symbol = cv2.resize(symbol, (w, h), interpolation=cv2.INTER_AREA)
        reshaped = np.zeros((60, 40), dtype=np.uint8)
        reshaped[30 - h + 10:40, 3:3 + w] = symbol
        reshaped = cv2.bitwise_not(reshaped)
        symbols.append(reshaped)

    logger.info(f"Preprocessing complete. Number of symbols: {len(symbols)}")
    return symbols

In [5]:
def load_data(data_dir):
    logger.info(f"Loading data from directory: {data_dir}")
    images = []
    labels = []
    for filename in os.listdir(data_dir):
        if filename.endswith(".png"):
            symbols = preprocess_image(os.path.join(data_dir, filename))
            # Добавьте метки для каждого символа
            image_labels = list(filename.split('.')[0])
            if len(symbols) != len(image_labels):
                logger.warning(f"Mismatch in symbols ({len(symbols)}) and labels ({len(image_labels)}) for {filename}")
                continue
            else:
                images.extend(symbols)
                labels.extend(image_labels)
    logger.info(f"Loaded {len(images)} images and {len(labels)} labels.")
    return np.array(images), np.array(labels)

In [6]:
images, labels = load_data('cpt_gen/generated')

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'cpt_gen/generated'

## Проверка количества изображений и меток

In [ ]:
assert len(images) == len(labels), f"Mismatch: {len(images)} images and {len(labels)} labels"

## Создание словаря для преобразования символов в индексы

In [ ]:
unique_labels = sorted(set(labels))
label_to_index = {label: index for index, label in enumerate(unique_labels)}
logger.debug(f"Unique labels: {unique_labels}")

## Преобразование меток в индексы

In [ ]:
labels_indices = np.array([label_to_index[label] for label in labels])

## Преобразование индексов в категориальные данные

In [ ]:
num_classes = len(unique_labels)
labels_categorical = to_categorical(labels_indices, num_classes)

## Нормализация изображений

In [ ]:
images = images / 255.0

## Изменение формы изображений для модели

In [ ]:
images = images.reshape((-1, 40, 60, 1))

## Создание модели

In [ ]:
logger.info("Creating the model.")
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(40, 60, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

C:\Users\ETB\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Компиляция модели

In [ ]:
logger.info("Compiling the model.")
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Обучение и сохранение модели

In [ ]:
logger.info("Training the model.")
model.fit(images, labels_categorical, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
684/684 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.7006 - loss: 1.2961 - val_accuracy: 0.9676 - val_loss: 0.1107
Epoch 2/10
684/684 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9737 - loss: 0.0840 - val_accuracy: 0.9751 - val_loss: 0.0656
Epoch 3/10
684/684 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.9786 - loss: 0.0571 - val_accuracy: 0.9808 - val_loss: 0.0677
Epoch 4/10
684/684 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9818 - loss: 0.0470 - val_accuracy: 0.9781 - val_loss: 0.0542
Epoch 5/10
684/684 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9839 - loss: 0.0439 - val_accuracy: 0.9793 - val_loss: 0.0623
Epoch 6/10
684/684 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9806 - loss: 0.0478 - val_accuracy: 0.9835 - val_loss: 0.0534
Epoch 7/10
684/684 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9815 - loss: 0.0428 - val_accuracy: 0.9830 - val_loss: 0.0555
Epoch 8/10
684/684 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9853 - loss: 0.0364 - val_accu

In [ ]:
logger.info("Saving the model.")
model.save('my_model_k.keras')

## Оценка модели

In [ ]:
logger.info("Evaluating the model.")
loss, accuracy = model.evaluate(images, labels_categorical)
logger.info(f'Loss: {loss}, Accuracy: {accuracy}')
print(f'Loss: {loss}, Accuracy: {accuracy}')


855/855 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9868 - loss: 0.0288
Loss: 0.03831688314676285, Accuracy: 0.9836905002593994
